# Pandas基础

### 七月在线python数据分析班 2017升级版 julyedu.com

pandas是一个专门用于数据分析的python library，pandas的开发基于numpy，所以pandas的各种功能和numpy很像，也是一个运行效率很高的library。

## [Pandas](http://pandas.pydata.org/)简介
- python数据分析library
- 基于numpy (对ndarray的操作)
- 有一种用python做Excel/SQL/R的感觉

## 目录
- Series
- DataFrame
- Index, reindex and hierarchical indexing
- Merge, Join, Concatenate, Groupby and Aggregate
- Read from csv
- 随堂案例：bikes routes counts

## 数据结构Series

### 七月在线python数据分析班 2017升级版 julyedu.com

### 构造和初始化Series

In [1]:
import pandas as pd
import numpy as np

Series是一个一维的数据结构

In [2]:
s = pd.Series([7, 'Beijing', 2.17, -12344, 'Happy Birthday!'])
s

0                  7
1            Beijing
2               2.17
3             -12344
4    Happy Birthday!
dtype: object

pandas会默认用0到n来作为Series的index，但是我们也可以自己指定index。

In [3]:
s = pd.Series([7, 'Beijing', 2.17, -12344, 'Happy Birthday!'],
             index=['A', 'B', 'C', 'D', 'E'])
s

A                  7
B            Beijing
C               2.17
D             -12344
E    Happy Birthday!
dtype: object

还可以用dictionary来构造一个Series，因为Series本来就是key value pairs。

In [5]:
cities = {'Beijing': 55000, 'Shanghai': 60000, 'Shenzhen': 50000, 'Hangzhou': 20000, 'Guangzhou': 25000, 'Suzhou': None}
apts = pd.Series(cities)
apts

Beijing      55000.0
Guangzhou    25000.0
Hangzhou     20000.0
Shanghai     60000.0
Shenzhen     50000.0
Suzhou           NaN
dtype: float64

### 选择数据

前面定义的index就是用来选择数据的

In [6]:
apts["Hangzhou"]

20000.0

In [7]:
apts[["Hangzhou", "Beijing", "Shenzhen"]]

Hangzhou    20000.0
Beijing     55000.0
Shenzhen    50000.0
dtype: float64

大家还记得上次numpy讲到的boolean indexing吗？pandas当中也可以使用。

In [8]:
apts[apts < 50000]

Guangzhou    25000.0
Hangzhou     20000.0
dtype: float64

下面我再详细展示一下这个boolean indexing是如何工作的

In [10]:
less_than_50000 = apts < 50000
print(less_than_50000)

Beijing      False
Guangzhou     True
Hangzhou      True
Shanghai     False
Shenzhen     False
Suzhou       False
dtype: bool


In [11]:
print(apts[less_than_50000])

Guangzhou    25000.0
Hangzhou     20000.0
dtype: float64


### Series元素赋值

Series的元素可以被赋值

In [13]:
print("Old value: ", apts['Shenzhen'])
apts['Shenzhen'] = 55000
print("New value: ", apts['Shenzhen'])

Old value:  40000.0
New value:  55000.0


前面讲过的boolean indexing在赋值的时候也可以用

In [14]:
print(apts[apts < 50000])
print()
apts[apts <= 50000] = 40000
print(apts[apts < 50000])

Guangzhou    40000.0
Hangzhou     40000.0
dtype: float64

Guangzhou    40000.0
Hangzhou     40000.0
dtype: float64


### 数学运算

下面我们来讲一些基本的数学运算。

In [15]:
apts / 2

Beijing      27500.0
Guangzhou    20000.0
Hangzhou     20000.0
Shanghai     30000.0
Shenzhen     27500.0
Suzhou           NaN
dtype: float64

In [16]:
np.square(apts)

Beijing      3.025000e+09
Guangzhou    1.600000e+09
Hangzhou     1.600000e+09
Shanghai     3.600000e+09
Shenzhen     3.025000e+09
Suzhou                NaN
dtype: float64

square也可以写成 **

In [19]:
apts ** 2

Beijing      3.025000e+09
Guangzhou    1.600000e+09
Hangzhou     1.600000e+09
Shanghai     3.600000e+09
Shenzhen     3.025000e+09
Suzhou                NaN
dtype: float64

我们再定义一个新的Series做加法

In [21]:
cars = pd.Series({'Beijing': 300000, 'Shanghai': 400000, 'Shenzhen': 300000, \
                      'Tianjin': 200000, 'Guangzhou': 200000, 'Chongqing': 150000})
cars

Beijing      300000
Chongqing    150000
Guangzhou    200000
Shanghai     400000
Shenzhen     300000
Tianjin      200000
dtype: int64

In [22]:
print(cars + apts * 100)

Beijing      5800000.0
Chongqing          NaN
Guangzhou    4200000.0
Hangzhou           NaN
Shanghai     6400000.0
Shenzhen     5800000.0
Suzhou             NaN
Tianjin            NaN
dtype: float64


### 数据缺失

In [23]:
print('Hangzhou' in apts)
print('Hangzhou' in cars)

True
False


In [24]:
apts.notnull()

Beijing       True
Guangzhou     True
Hangzhou      True
Shanghai      True
Shenzhen      True
Suzhou       False
dtype: bool

In [29]:
print(apts.isnull())
print()

Beijing      False
Guangzhou    False
Hangzhou     False
Shanghai     False
Shenzhen     False
Suzhou        True
dtype: bool



In [30]:
print(apts[apts.isnull()])

Suzhou   NaN
dtype: float64


In [31]:

print(apts[apts.isnull() == False])

Beijing      55000.0
Guangzhou    40000.0
Hangzhou     40000.0
Shanghai     60000.0
Shenzhen     55000.0
dtype: float64


## 数据结构[Dataframe](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html)

### 七月在线python数据分析班 2017升级版 julyedu.com

一个Dataframe就是一张表格，Series表示的是一维数组，Dataframe则是一个二维数组，可以类比成一张excel的spreadsheet。也可以把Dataframe当做一组Series的集合。

### 创建一个DataFrame

dataframe可以由一个dictionary构造得到。

In [33]:
data = {'city': ['Beijing', 'Shanghai', 'Guangzhou', 'Shenzhen', 'Hangzhou', 'Chongqing'],
       'year': [2016,2017,2016,2017,2016, 2016],
       'population': [2100, 2300, 1000, 700, 500, 500]}
print(pd.DataFrame(data))

        city  population  year
0    Beijing        2100  2016
1   Shanghai        2300  2017
2  Guangzhou        1000  2016
3   Shenzhen         700  2017
4   Hangzhou         500  2016
5  Chongqing         500  2016


columns的名字和顺序可以指定

In [34]:
print(pd.DataFrame(data, columns=['year', 'city', 'population']))

   year       city  population
0  2016    Beijing        2100
1  2017   Shanghai        2300
2  2016  Guangzhou        1000
3  2017   Shenzhen         700
4  2016   Hangzhou         500
5  2016  Chongqing         500


In [37]:
frame2 = pd.DataFrame(data, \
                     columns = ['year', 'city', 'population', 'debt'],
                     index = ['one', 'two', 'three', 'four', 'five', 'six'])
print(frame2)

       year       city  population debt
one    2016    Beijing        2100  NaN
two    2017   Shanghai        2300  NaN
three  2016  Guangzhou        1000  NaN
four   2017   Shenzhen         700  NaN
five   2016   Hangzhou         500  NaN
six    2016  Chongqing         500  NaN


### 从DataFrame里选择数据

In [44]:
print(frame2['city'])
type(frame2['city'])

one        Beijing
two       Shanghai
three    Guangzhou
four      Shenzhen
five      Hangzhou
six      Chongqing
Name: city, dtype: object


pandas.core.series.Series

In [45]:
print(frame2.year)
type(frame2.year)

one      2016
two      2017
three    2016
four     2017
five     2016
six      2016
Name: year, dtype: int64


pandas.core.series.Series

In [58]:
print(frame2.ix['three'])
type(frame2.ix['three'])

year               2016
city          Guangzhou
population         1000
debt                100
Name: three, dtype: object


pandas.core.series.Series

下面这种方法默认用来选列而不是选行

In [60]:
# print(frame2["three"])

In [52]:
print(frame2.ix[1])
type(frame2.ix[1])

year              2017
city          Shanghai
population        2300
debt               NaN
Name: two, dtype: object


pandas.core.series.Series

### DataFrame元素赋值

In [71]:
frame2["population"]["one"] = 2100

c:\users\jasonchuzewei\anaconda3\envs\julyedu\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


可以给一整列赋值

In [70]:
frame2['debt'] = 100
print(frame2)

       year       city  population  debt western
one    2016    Beijing        2200   100   False
two    2017   Shanghai        2300   100   False
three  2016  Guangzhou        1000   100   False
four   2017   Shenzhen         700   100   False
five   2016   Hangzhou         500   100   False
six    2016  Chongqing         500   100    True


In [73]:
frame2.ix['six'] = 0
print(frame2)

       year       city  population  debt western
one    2016    Beijing        2100     0   False
two    2017   Shanghai        2300     1   False
three  2016  Guangzhou        1000     2   False
four   2017   Shenzhen         700     3   False
five   2016   Hangzhou         500     4   False
six       0          0           0     0   False


In [74]:
frame2 = pd.DataFrame(data, \
                     columns = ['year', 'city', 'population', 'debt'],
                     index = ['one', 'two', 'three', 'four', 'five', 'six'])
print(frame2)

       year       city  population debt
one    2016    Beijing        2100  NaN
two    2017   Shanghai        2300  NaN
three  2016  Guangzhou        1000  NaN
four   2017   Shenzhen         700  NaN
five   2016   Hangzhou         500  NaN
six    2016  Chongqing         500  NaN


In [107]:
frame2
frame2.index
frame2['city']

one        Beijing
two       Shanghai
three    Guangzhou
four      Shenzhen
five      Hangzhou
six      Chongqing
Name: city, dtype: object

In [75]:
frame2.debt = np.arange(6)
print(frame2)

       year       city  population  debt
one    2016    Beijing        2100     0
two    2017   Shanghai        2300     1
three  2016  Guangzhou        1000     2
four   2017   Shenzhen         700     3
five   2016   Hangzhou         500     4
six    2016  Chongqing         500     5


还可以用Series来指定需要修改的index以及相对应的value，没有指定的默认用NaN.

In [76]:
val = pd.Series([100, 200, 300], index=['two', 'three', 'five'])
frame2['debt'] = val
print(frame2)

       year       city  population   debt
one    2016    Beijing        2100    NaN
two    2017   Shanghai        2300  100.0
three  2016  Guangzhou        1000  200.0
four   2017   Shenzhen         700    NaN
five   2016   Hangzhou         500  300.0
six    2016  Chongqing         500    NaN


In [77]:
frame2['western'] = (frame2.city == 'Chongqing')
print(frame2)

       year       city  population   debt western
one    2016    Beijing        2100    NaN   False
two    2017   Shanghai        2300  100.0   False
three  2016  Guangzhou        1000  200.0   False
four   2017   Shenzhen         700    NaN   False
five   2016   Hangzhou         500  300.0   False
six    2016  Chongqing         500    NaN    True


In [81]:
print(frame2.columns)

Index(['year', 'city', 'population', 'debt', 'western'], dtype='object')


一个DataFrame就和一个numpy 2d array一样，可以被转置

In [82]:
pop = {'Beijing': {2016: 2100, 2017:2200},
      'Shanghai': {2015:2400, 2016:2500, 2017:2600}}

In [83]:
frame3 = pd.DataFrame(pop)
print(frame3)
print(frame3.T)

      Beijing  Shanghai
2015      NaN      2400
2016   2100.0      2500
2017   2200.0      2600
            2015    2016    2017
Beijing      NaN  2100.0  2200.0
Shanghai  2400.0  2500.0  2600.0


指定index的顺序，以及使用切片初始化数据

In [85]:
print(pd.DataFrame(pop, index=[2016,2015,2017]))

      Beijing  Shanghai
2016   2100.0      2500
2015      NaN      2400
2017   2200.0      2600


In [86]:
pdata = {'Beijing': frame3['Beijing'][:-1], 'Shanghai':frame3['Shanghai'][:-1]}
print(pd.DataFrame(pdata))

      Beijing  Shanghai
2015      NaN      2400
2016   2100.0      2500


我们还可以指定index的名字和列的名字

In [87]:
frame3.index.name = 'year'
frame3.columns.name = 'city'
print(frame3)

city  Beijing  Shanghai
year                   
2015      NaN      2400
2016   2100.0      2500
2017   2200.0      2600


In [88]:
print(frame3.values)
print(frame2)
print(type(frame2.values))

[[   nan  2400.]
 [ 2100.  2500.]
 [ 2200.  2600.]]
       year       city  population   debt western
one    2016    Beijing        2100    NaN   False
two    2017   Shanghai        2300  100.0   False
three  2016  Guangzhou        1000  200.0   False
four   2017   Shenzhen         700    NaN   False
five   2016   Hangzhou         500  300.0   False
six    2016  Chongqing         500    NaN    True
<class 'numpy.ndarray'>


## Index
### 七月在线python数据分析班 2017升级版 julyedu.com

### index object

In [90]:
obj = pd.Series(range(3), index = ['a', 'b', 'c'])
index = obj.index
print(index)
print(index[1:])

Index(['a', 'b', 'c'], dtype='object')
Index(['b', 'c'], dtype='object')


index的值是不能被更改的

In [92]:
index[1] = 'd'

TypeError: Index does not support mutable operations

In [98]:
index = pd.Index(np.arange(3))
obj2 = pd.Series([2,5,7], index=index)
print(obj2)
print(obj2.index is index)
print(obj2.index is np.arange(3))

0    2
1    5
2    7
dtype: int64
True
False


In [96]:
pop = {'Beijing': {2016: 2100, 2017:2200},
      'Shanghai': {2015:2400, 2016:2500, 2017:2600}}
frame3 = pd.DataFrame(pop)
print('Shanghai' in frame3.columns)
print(2015 in frame3.index)

True
True


### 针对index进行索引和切片

In [99]:
obj = pd.Series(np.arange(4), index=['a', 'b', 'c', 'd'])
print(obj['b'])

1


默认的数字index依旧可以使用

In [103]:
print(obj[3])
print()
print(obj[[1,3]])

3

b    1
d    3
dtype: int32


In [30]:
print(obj[obj<2])

a    0
b    1
dtype: int32


下面介绍如何对Series进行切片

In [31]:
print(obj['b':'c'])
obj['b':'c'] = 5
print(obj)

b    1
c    2
dtype: int32
a    0
b    5
c    5
d    3
dtype: int32


对DataFrame进行Indexing与Series基本相同

In [110]:
frame = pd.DataFrame(np.arange(9).reshape(3,3), 
                    index = ['a', 'c', 'd'],
                    columns = ['Hangzhou', 'Shenzhen', 'Nanjing'])

In [35]:
print(frame)

   Hangzhou  Shenzhen  Nanjing
a         0         1        2
c         3         4        5
d         6         7        8


In [37]:
print(frame['Hangzhou'])

a    0
c    3
d    6
Name: Hangzhou, dtype: int32


In [39]:
print(frame[['Shenzhen', 'Nanjing']])

   Shenzhen  Nanjing
a         1        2
c         4        5
d         7        8


In [114]:
print(frame[:2])

   Hangzhou  Shenzhen  Nanjing
a         0         1        2
c         3         4        5


In [115]:
print(frame.ix['a'])

Hangzhou    0
Shenzhen    1
Nanjing     2
Name: a, dtype: int32


In [116]:
print(frame.ix[['a','d'], ['Shenzhen', 'Nanjing']])

   Shenzhen  Nanjing
a         1        2
d         7        8


In [123]:
print(frame.ix[:'c', 'Hangzhou'])

a    0
c    3
Name: Hangzhou, dtype: int32


DataFrame也可以用condition selection

In [124]:
print(frame[frame.Hangzhou > 1])

   Hangzhou  Shenzhen  Nanjing
c         3         4        5
d         6         7        8


In [125]:
print(frame < 5)

  Hangzhou Shenzhen Nanjing
a     True     True    True
c     True     True   False
d    False    False   False


In [126]:
frame[frame < 5] = 0
print(frame)

   Hangzhou  Shenzhen  Nanjing
a         0         0        0
c         0         0        5
d         6         7        8


### [reindex](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.reindex.html)

把一个Series或者DataFrame按照新的index顺序进行重排

In [3]:
import numpy as np
import pandas as pd

In [127]:
obj = pd.Series([4.5, 7.2, -5.3, 3.2], index=['d', 'b', 'a', 'c'])
print(obj)

d    4.5
b    7.2
a   -5.3
c    3.2
dtype: float64


In [128]:
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])
print(obj2)

a   -5.3
b    7.2
c    3.2
d    4.5
e    NaN
dtype: float64


In [129]:
print(obj.reindex(['a', 'b', 'c', 'd', 'e'], fill_value = 0))

a   -5.3
b    7.2
c    3.2
d    4.5
e    0.0
dtype: float64


In [130]:
obj3 = pd.Series(['blue', 'purple', 'yellow'], index = [0,2,4])
print(obj3)

0      blue
2    purple
4    yellow
dtype: object


In [131]:
print(obj3.reindex(range(6), method='ffill'))

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object


In [132]:
print(obj3.reindex(range(6), method='bfill'))

0      blue
1    purple
2    purple
3    yellow
4    yellow
5       NaN
dtype: object


既然我们可以对Series进行reindex，相应地，我们也可以用同样的方法对DataFrame进行reindex。

In [133]:
frame = pd.DataFrame(np.arange(9).reshape(3,3), 
                    index = ['a', 'c', 'd'],
                    columns = ['Hangzhou', 'Shenzhen', 'Nanjing'])
print(frame)

   Hangzhou  Shenzhen  Nanjing
a         0         1        2
c         3         4        5
d         6         7        8


In [134]:
frame2 = frame.reindex(['a', 'b', 'c', 'd'])
print(frame2)

   Hangzhou  Shenzhen  Nanjing
a       0.0       1.0      2.0
b       NaN       NaN      NaN
c       3.0       4.0      5.0
d       6.0       7.0      8.0


在reindex的同时，我们还可以重新指定columns

In [136]:
print(frame.reindex(columns = ['Shenzhen', 'Hangzhou', 'Chongqing']))

   Shenzhen  Hangzhou  Chongqing
a         1         0        NaN
c         4         3        NaN
d         7         6        NaN


In [137]:
print(frame.reindex(index = ['a', 'b', 'c', 'd'],
                    method = 'ffill',
                    columns = ['Shenzhen', 'Hangzhou', 'Chongqing']))
print(frame.ix[['a', 'b', 'd', 'c'], ['Shenzhen', 'Hangzhou', 'Chongqing']])

   Shenzhen  Hangzhou  Chongqing
a         1         0        NaN
b         1         0        NaN
c         4         3        NaN
d         7         6        NaN
   Shenzhen  Hangzhou  Chongqing
a       1.0       0.0        NaN
b       NaN       NaN        NaN
d       7.0       6.0        NaN
c       4.0       3.0        NaN


In [ ]:
下面介绍如何用drop来删除Series和DataFrame中的index

In [138]:
print(obj3)
obj4 = obj3.drop(2)
print(obj4)

0      blue
2    purple
4    yellow
dtype: object
0      blue
4    yellow
dtype: object


In [139]:
print(obj3.drop([2,4]))

0    blue
dtype: object


In [140]:
print(frame)

   Hangzhou  Shenzhen  Nanjing
a         0         1        2
c         3         4        5
d         6         7        8


In [141]:
print(frame.drop(['a', 'c']))

   Hangzhou  Shenzhen  Nanjing
d         6         7        8


drop不仅仅可以删除行，还可以删除列

In [142]:
print(frame.drop('Shenzhen', axis=1))

   Hangzhou  Nanjing
a         0        2
c         3        5
d         6        8


In [143]:
print(frame.drop(['Shenzhen', 'Hangzhou'], axis=1))

   Nanjing
a        2
c        5
d        8


### hierarchical index

In [2]:
import numpy as np
import pandas as pd

Series的hierarchical indexing

In [144]:
data = pd.Series(np.random.randn(10), index=[['a','a','a','b','b','c','c','c','d','d'], [1,2,3,1,2,1,2,3,1,2]])
print(data)

a  1   -0.370285
   2    0.509911
   3    0.845561
b  1   -0.308466
   2   -0.493243
c  1    1.799576
   2    2.469641
   3    1.150197
d  1    0.802092
   2   -0.356297
dtype: float64


In [145]:
print(data.index)

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 2, 2, 2, 3, 3], [0, 1, 2, 0, 1, 0, 1, 2, 0, 1]])


In [146]:
print(data.b)

1   -0.308466
2   -0.493243
dtype: float64


In [7]:
print(data['b':'c'])

b  1    0.641165
   2    0.174197
c  1   -0.368307
   2    2.280507
   3    0.683652
dtype: float64


In [8]:
print(data[:2])

a  1   -0.571858
   2   -0.542655
dtype: float64


unstack和stack可以帮助我们在hierarchical indexing和DataFrame之间进行切换。

In [147]:
print(data.unstack())
print(type(data.unstack()))

          1         2         3
a -0.370285  0.509911  0.845561
b -0.308466 -0.493243       NaN
c  1.799576  2.469641  1.150197
d  0.802092 -0.356297       NaN
<class 'pandas.core.frame.DataFrame'>


In [148]:
print(data.unstack().stack())
print(type(data.unstack().stack()))

a  1   -0.370285
   2    0.509911
   3    0.845561
b  1   -0.308466
   2   -0.493243
c  1    1.799576
   2    2.469641
   3    1.150197
d  1    0.802092
   2   -0.356297
dtype: float64
<class 'pandas.core.series.Series'>


DataFrame的hierarchical indexing

In [149]:
frame = pd.DataFrame(np.arange(12).reshape((4,3)),
                    index = [['a','a','b','b'], [1,2,1,2]],
                    columns = [['Beijing', 'Beijing', 'Shanghai'], ['apts', 'cars', 'apts']])
print(frame)

    Beijing      Shanghai
       apts cars     apts
a 1       0    1        2
  2       3    4        5
b 1       6    7        8
  2       9   10       11


In [150]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['city', 'type']
print(frame)

city      Beijing      Shanghai
type         apts cars     apts
key1 key2                      
a    1          0    1        2
     2          3    4        5
b    1          6    7        8
     2          9   10       11


In [154]:
print(frame.ix['a', 1])
print(type(frame.ix['a', 1]))

city      type
Beijing   apts    0
          cars    1
Shanghai  apts    2
Name: (a, 1), dtype: int32
<class 'pandas.core.series.Series'>


In [155]:
print(frame.ix['a', 2]['Beijing'])

type
apts    3
cars    4
Name: (a, 2), dtype: int32


In [156]:
print(frame.ix['a', 2]['Beijing']['apts'])

3


## 关于Merge, Join和Concatenate

### 七月在线python数据分析班 2017升级版 julyedu.com

In [158]:
df1 = pd.DataFrame({'apts': [55000, 60000],
                   'cars': [200000, 300000],},
                  index = ['Shanghai', 'Beijing'])
print(df1)

           apts    cars
Shanghai  55000  200000
Beijing   60000  300000


In [159]:
df2 = pd.DataFrame({'apts': [25000, 20000],
                   'cars': [150000, 120000],},
                  index = ['Hangzhou', 'Najing'])
print(df2)

           apts    cars
Hangzhou  25000  150000
Najing    20000  120000


In [160]:
df3 = pd.DataFrame({'apts': [30000, 10000],
                   'cars': [180000, 100000],},
                  index = ['Guangzhou', 'Chongqing'])
print(df3)

            apts    cars
Guangzhou  30000  180000
Chongqing  10000  100000


### concatenate

In [161]:
frames = [df1, df2, df3]
result = pd.concat(frames)
print(result)

            apts    cars
Shanghai   55000  200000
Beijing    60000  300000
Hangzhou   25000  150000
Najing     20000  120000
Guangzhou  30000  180000
Chongqing  10000  100000


在concatenate的时候可以指定keys，这样可以给每一个部分加上一个Key。

以下的例子就构造了一个hierarchical index。

In [162]:
result2 = pd.concat(frames, keys=['x', 'y', 'z'])
print(result2)

              apts    cars
x Shanghai   55000  200000
  Beijing    60000  300000
y Hangzhou   25000  150000
  Najing     20000  120000
z Guangzhou  30000  180000
  Chongqing  10000  100000


In [163]:
result2.ix['y']

apts    cars
Hangzhou  25000  150000
Najing    20000  120000

In [164]:
df4 = pd.DataFrame({'salaries': [10000, 30000, 30000, 20000, 15000]},
                  index = ['Suzhou', 'Beijing', 'Shanghai', 'Guangzhou', 'Tianjin'])
print(df4)

           salaries
Suzhou        10000
Beijing       30000
Shanghai      30000
Guangzhou     20000
Tianjin       15000


In [165]:
result3 = pd.concat([result, df4], axis=1)
print(result3)

              apts      cars  salaries
Beijing    60000.0  300000.0   30000.0
Chongqing  10000.0  100000.0       NaN
Guangzhou  30000.0  180000.0   20000.0
Hangzhou   25000.0  150000.0       NaN
Najing     20000.0  120000.0       NaN
Shanghai   55000.0  200000.0   30000.0
Suzhou         NaN       NaN   10000.0
Tianjin        NaN       NaN   15000.0


复习一下前面讲过的stack

In [168]:
print(result3.stack())
print(type(result3.stack()))

Beijing    apts         60000.0
           cars        300000.0
           salaries     30000.0
Chongqing  apts         10000.0
           cars        100000.0
Guangzhou  apts         30000.0
           cars        180000.0
           salaries     20000.0
Hangzhou   apts         25000.0
           cars        150000.0
Najing     apts         20000.0
           cars        120000.0
Shanghai   apts         55000.0
           cars        200000.0
           salaries     30000.0
Suzhou     salaries     10000.0
Tianjin    salaries     15000.0
dtype: float64
<class 'pandas.core.series.Series'>


用inner可以去掉NaN

In [169]:
result3 = pd.concat([result, df4], axis=1, join='inner')
print(result3)

            apts    cars  salaries
Beijing    60000  300000     30000
Shanghai   55000  200000     30000
Guangzhou  30000  180000     20000


#### 用append来做concatenation

In [170]:
print(df1.append(df2))

           apts    cars
Shanghai  55000  200000
Beijing   60000  300000
Hangzhou  25000  150000
Najing    20000  120000


In [171]:
print(df1.append(df4))

              apts      cars  salaries
Shanghai   55000.0  200000.0       NaN
Beijing    60000.0  300000.0       NaN
Suzhou         NaN       NaN   10000.0
Beijing        NaN       NaN   30000.0
Shanghai       NaN       NaN   30000.0
Guangzhou      NaN       NaN   20000.0
Tianjin        NaN       NaN   15000.0


Series和DataFrame还可以被一起concatenate，这时候Series会先被转成DataFrame然后做Join，因为Series本来就是一个只有一维的DataFrame对吧。

In [172]:
s1 = pd.Series([60, 50], index=['Shanghai', 'Beijing'], name='meal')
print(s1)

Shanghai    60
Beijing     50
Name: meal, dtype: int64


In [174]:
print(df1)

           apts    cars
Shanghai  55000  200000
Beijing   60000  300000


In [175]:
print(pd.concat([df1, s1], axis=1))

           apts    cars  meal
Shanghai  55000  200000    60
Beijing   60000  300000    50


下面讲如何append一个row到DataFrame里。

In [176]:
s2 = pd.Series([18000, 12000], index=['apts', 'cars'], name='Xiamen') #注意这里的name是必须要有的，因为要用作Index。
print(s2)

apts    18000
cars    12000
Name: Xiamen, dtype: int64


In [177]:
print(df1.append(s2))

           apts    cars
Shanghai  55000  200000
Beijing   60000  300000
Xiamen    18000   12000


### Merge(Join)
### 七月在线python数据分析班 2017升级版 julyedu.com

In [179]:
df1 = pd.DataFrame({'apts': [55000, 60000, 58000],
                   'cars': [200000, 300000,250000],
                  'cities': ['Shanghai', 'Beijing','Shenzhen']})
print(df1)

    apts    cars    cities
0  55000  200000  Shanghai
1  60000  300000   Beijing
2  58000  250000  Shenzhen


In [180]:
df4 = pd.DataFrame({'salaries': [10000, 30000, 30000, 20000, 15000],
                  'cities': ['Suzhou', 'Beijing', 'Shanghai', 'Guangzhou', 'Tianjin']})
print(df4)

      cities  salaries
0     Suzhou     10000
1    Beijing     30000
2   Shanghai     30000
3  Guangzhou     20000
4    Tianjin     15000


In [181]:
result = pd.merge(df1, df4, on='cities')
print(result)

    apts    cars    cities  salaries
0  55000  200000  Shanghai     30000
1  60000  300000   Beijing     30000


In [182]:
result = pd.merge(df1, df4, on='cities', how='outer')
print(result)

      apts      cars     cities  salaries
0  55000.0  200000.0   Shanghai   30000.0
1  60000.0  300000.0    Beijing   30000.0
2  58000.0  250000.0   Shenzhen       NaN
3      NaN       NaN     Suzhou   10000.0
4      NaN       NaN  Guangzhou   20000.0
5      NaN       NaN    Tianjin   15000.0


In [183]:
result = pd.merge(df1, df4, on='cities', how='right')
print(result)

      apts      cars     cities  salaries
0  55000.0  200000.0   Shanghai     30000
1  60000.0  300000.0    Beijing     30000
2      NaN       NaN     Suzhou     10000
3      NaN       NaN  Guangzhou     20000
4      NaN       NaN    Tianjin     15000


In [184]:
result = pd.merge(df1, df4, on='cities', how='left')
print(result)

    apts    cars    cities  salaries
0  55000  200000  Shanghai   30000.0
1  60000  300000   Beijing   30000.0
2  58000  250000  Shenzhen       NaN


#### join on index

In [185]:
df1 = pd.DataFrame({'apts': [55000, 60000, 58000],
                   'cars': [200000, 300000,250000]},
                  index=['Shanghai', 'Beijing','Shenzhen'])
print(df1)

           apts    cars
Shanghai  55000  200000
Beijing   60000  300000
Shenzhen  58000  250000


In [186]:
df4 = pd.DataFrame({'salaries': [10000, 30000, 30000, 20000, 15000]},
                  index=['Suzhou', 'Beijing', 'Shanghai', 'Guangzhou', 'Tianjin'])
print(df4)

           salaries
Suzhou        10000
Beijing       30000
Shanghai      30000
Guangzhou     20000
Tianjin       15000


In [187]:
print(df1.join(df4))

           apts    cars  salaries
Shanghai  55000  200000   30000.0
Beijing   60000  300000   30000.0
Shenzhen  58000  250000       NaN


In [188]:
print(df1.join(df4, how='outer'))

              apts      cars  salaries
Beijing    60000.0  300000.0   30000.0
Guangzhou      NaN       NaN   20000.0
Shanghai   55000.0  200000.0   30000.0
Shenzhen   58000.0  250000.0       NaN
Suzhou         NaN       NaN   10000.0
Tianjin        NaN       NaN   15000.0


也可以用merge来写

In [189]:
print(pd.merge(df1, df4, left_index=True, right_index=True, how='outer'))

              apts      cars  salaries
Beijing    60000.0  300000.0   30000.0
Guangzhou      NaN       NaN   20000.0
Shanghai   55000.0  200000.0   30000.0
Shenzhen   58000.0  250000.0       NaN
Suzhou         NaN       NaN   10000.0
Tianjin        NaN       NaN   15000.0


## Group By
### 七月在线python数据分析班 2017升级版 julyedu.com

举个栗子，假设我们现在有一张公司每个员工的收入流水。

In [191]:
import pandas as pd
salaries = pd.DataFrame({
    'Name': ['July', 'Chu', 'Chu', 'Lin', 'July', 'July', 'Chu', 'July'],
    'Year': [2016,2016,2016,2016,2017,2017,2017,2017],
    'Salary': [10000,2000,4000,5000,18000,25000,3000,4000],
    'Bonus': [3000,1000,1000,1200,4000,2300,500,1000]
})
print(salaries)

   Bonus  Name  Salary  Year
0   3000  July   10000  2016
1   1000   Chu    2000  2016
2   1000   Chu    4000  2016
3   1200   Lin    5000  2016
4   4000  July   18000  2017
5   2300  July   25000  2017
6    500   Chu    3000  2017
7   1000  July    4000  2017


接下来我给大家演示一下什么叫做Group By

In [192]:
group_by_name = salaries.groupby('Name')

### groupby经常和aggregate一起使用

In [193]:
group_by_name.aggregate(sum)

Bonus  Salary  Year
Name                     
Chu    2500    9000  6049
July  10300   57000  8067
Lin    1200    5000  2016

也可以这么写

In [194]:
group_by_name.sum()

Bonus  Salary  Year
Name                     
Chu    2500    9000  6049
July  10300   57000  8067
Lin    1200    5000  2016

In [195]:
group_by_name_year = salaries.groupby(['Name', 'Year'])
group_by_name_year.sum()

Bonus  Salary
Name Year               
Chu  2016   2000    6000
     2017    500    3000
July 2016   3000   10000
     2017   7300   47000
Lin  2016   1200    5000

In [196]:
group_by_name_year.size()

Name  Year
Chu   2016    2
      2017    1
July  2016    1
      2017    3
Lin   2016    1
dtype: int64

describe这个function可以为我们展示各种有用的统计信息

In [197]:
# group_by_name_year.describe()

## READ FROM CSV
### 七月在线python数据分析班 2017升级版 julyedu.com

In [199]:
import pandas as pd

我们先从CSV文件中读取一些数据。

bike.csv记录了Montreal自行车路线的数据，具体有7条路线，数据记录了每条自行车路线每天分别有多少人。

In [200]:
bikes = pd.read_csv('bikes.csv', encoding='latin1')
# bikes

In [201]:
bikes = pd.read_csv('bikes.csv', sep=';', parse_dates=['Date'], encoding='latin1', dayfirst=True, index_col='Date')
# bikes

更多关于read_csv的说明请查阅documentation

[read_csv](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html)

In [202]:
bikes.head()

Berri 1  Brébeuf (données non disponibles)  Côte-Sainte-Catherine  \
Date                                                                            
2012-01-01       35                                NaN                      0   
2012-01-02       83                                NaN                      1   
2012-01-03      135                                NaN                      2   
2012-01-04      144                                NaN                      1   
2012-01-05      197                                NaN                      2   

            Maisonneuve 1  Maisonneuve 2  du Parc  Pierre-Dupuy  Rachel1  \
Date                                                                       
2012-01-01             38             51       26            10       16   
2012-01-02             68            153       53             6       43   
2012-01-03            104            248       89             3       58   
2012-01-04            116            318      111             8       61   
2012-01-05            124            330       97            13       95   

            St-Urbain (données non disponibles)  
Date                                             
2012-01-01                                  NaN  
2012-01-02                                  NaN  
2012-01-03                                  NaN  
2012-01-04                                  NaN  
2012-01-05                                  NaN

dropna会删除所有带NA的行

In [203]:
bikes.dropna()

Empty DataFrame
Columns: [Berri 1, Brébeuf (données non disponibles), Côte-Sainte-Catherine, Maisonneuve 1, Maisonneuve 2, du Parc, Pierre-Dupuy, Rachel1, St-Urbain (données non disponibles)]
Index: []

In [204]:
bikes.dropna(how='all').head()

Berri 1  Brébeuf (données non disponibles)  Côte-Sainte-Catherine  \
Date                                                                            
2012-01-01       35                                NaN                      0   
2012-01-02       83                                NaN                      1   
2012-01-03      135                                NaN                      2   
2012-01-04      144                                NaN                      1   
2012-01-05      197                                NaN                      2   

            Maisonneuve 1  Maisonneuve 2  du Parc  Pierre-Dupuy  Rachel1  \
Date                                                                       
2012-01-01             38             51       26            10       16   
2012-01-02             68            153       53             6       43   
2012-01-03            104            248       89             3       58   
2012-01-04            116            318      111             8       61   
2012-01-05            124            330       97            13       95   

            St-Urbain (données non disponibles)  
Date                                             
2012-01-01                                  NaN  
2012-01-02                                  NaN  
2012-01-03                                  NaN  
2012-01-04                                  NaN  
2012-01-05                                  NaN

In [205]:
bikes.dropna(axis=1, how='all').head()

Berri 1  Côte-Sainte-Catherine  Maisonneuve 1  Maisonneuve 2  \
Date                                                                       
2012-01-01       35                      0             38             51   
2012-01-02       83                      1             68            153   
2012-01-03      135                      2            104            248   
2012-01-04      144                      1            116            318   
2012-01-05      197                      2            124            330   

            du Parc  Pierre-Dupuy  Rachel1  
Date                                        
2012-01-01       26            10       16  
2012-01-02       53             6       43  
2012-01-03       89             3       58  
2012-01-04      111             8       61  
2012-01-05       97            13       95

下面给大家介绍如何填充缺失的数据

In [207]:
row = bikes.ix[0].copy()
row.fillna(row.mean())

Berri 1                                35.000000
Brébeuf (données non disponibles)      25.142857
Côte-Sainte-Catherine                   0.000000
Maisonneuve 1                          38.000000
Maisonneuve 2                          51.000000
du Parc                                26.000000
Pierre-Dupuy                           10.000000
Rachel1                                16.000000
St-Urbain (données non disponibles)    25.142857
Name: 2012-01-01 00:00:00, dtype: float64

In [215]:
m = bikes.mean(axis=1)
for i, col in enumerate(bikes):
    bikes.ix[:, i] = bikes.ix[:, i].fillna(m)
    
bikes.head()

Berri 1  Brébeuf (données non disponibles)  Côte-Sainte-Catherine  \
Date                                                                            
2012-01-01       35                          25.142857                      0   
2012-01-02       83                          58.142857                      1   
2012-01-03      135                          91.285714                      2   
2012-01-04      144                         108.428571                      1   
2012-01-05      197                         122.571429                      2   

            Maisonneuve 1  Maisonneuve 2  du Parc  Pierre-Dupuy  Rachel1  \
Date                                                                       
2012-01-01             38             51       26            10       16   
2012-01-02             68            153       53             6       43   
2012-01-03            104            248       89             3       58   
2012-01-04            116            318      111             8       61   
2012-01-05            124            330       97            13       95   

            St-Urbain (données non disponibles)  
Date                                             
2012-01-01                            25.142857  
2012-01-02                            58.142857  
2012-01-03                            91.285714  
2012-01-04                           108.428571  
2012-01-05                           122.571429

In [216]:
bikes[:5]

Berri 1  Brébeuf (données non disponibles)  Côte-Sainte-Catherine  \
Date                                                                            
2012-01-01       35                          25.142857                      0   
2012-01-02       83                          58.142857                      1   
2012-01-03      135                          91.285714                      2   
2012-01-04      144                         108.428571                      1   
2012-01-05      197                         122.571429                      2   

            Maisonneuve 1  Maisonneuve 2  du Parc  Pierre-Dupuy  Rachel1  \
Date                                                                       
2012-01-01             38             51       26            10       16   
2012-01-02             68            153       53             6       43   
2012-01-03            104            248       89             3       58   
2012-01-04            116            318      111             8       61   
2012-01-05            124            330       97            13       95   

            St-Urbain (données non disponibles)  
Date                                             
2012-01-01                            25.142857  
2012-01-02                            58.142857  
2012-01-03                            91.285714  
2012-01-04                           108.428571  
2012-01-05                           122.571429

In [217]:
berri_bikes = bikes[['Berri 1']].copy()
berri_bikes.head()

Berri 1
Date               
2012-01-01       35
2012-01-02       83
2012-01-03      135
2012-01-04      144
2012-01-05      197

In [218]:
berri_bikes.index

DatetimeIndex(['2012-01-01', '2012-01-02', '2012-01-03', '2012-01-04',
               '2012-01-05', '2012-01-06', '2012-01-07', '2012-01-08',
               '2012-01-09', '2012-01-10',
               ...
               '2012-10-27', '2012-10-28', '2012-10-29', '2012-10-30',
               '2012-10-31', '2012-11-01', '2012-11-02', '2012-11-03',
               '2012-11-04', '2012-11-05'],
              dtype='datetime64[ns]', name='Date', length=310, freq=None)

In [220]:
berri_bikes.index.weekday

array([6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0,
       1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2,
       3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4,
       5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6,
       0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1,
       2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3,
       4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5,
       6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0,
       1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2,
       3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4,
       5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6,
       0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1,
       2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3,
       4, 5,

In [221]:
berri_bikes.loc[:, 'weekday'] = berri_bikes.index.weekday
berri_bikes[:5]

Berri 1  weekday
Date                        
2012-01-01       35        6
2012-01-02       83        0
2012-01-03      135        1
2012-01-04      144        2
2012-01-05      197        3

有了weekday信息之后，我们就可以用上我们前面学过的.groupyby把骑车人数按照weekday分类，然后用aggregate算出每个工作日的骑车人数之和。

In [222]:
weekday_counts = berri_bikes.groupby('weekday').aggregate(sum)
weekday_counts

Berri 1
weekday         
0         134298
1         135305
2         152972
3         160131
4         141771
5         101578
6          99310

In [232]:
weekday_counts.index = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weekday_counts

Berri 1
Monday      134298
Tuesday     135305
Wednesday   152972
Thursday    160131
Friday      141771
Saturday    101578
Sunday       99310

接下来我们试试能不能把每条路线都加起来，然后算出一天骑自行车出门的人数之和。

In [236]:
bikes_sum = bikes.sum(axis=1).to_frame()
bikes_sum.head()
# bikes_sum.index

0
Date                   
2012-01-01   226.285714
2012-01-02   523.285714
2012-01-03   821.571429
2012-01-04   975.857143
2012-01-05  1103.142857

In [237]:
bikes_sum.ix[:, 'weekday'] = bikes_sum.index.weekday
bikes_sum.head()
#type(berri_bikes)

0  weekday
Date                            
2012-01-01   226.285714        6
2012-01-02   523.285714        0
2012-01-03   821.571429        1
2012-01-04   975.857143        2
2012-01-05  1103.142857        3

In [238]:
weekday_counts = bikes_sum.groupby('weekday').aggregate(sum)
weekday_counts.index = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weekday_counts

0
Monday     9.192381e+05
Tuesday    8.981769e+05
Wednesday  1.015357e+06
Thursday   1.065946e+06
Friday     9.498497e+05
Saturday   6.643299e+05
Sunday     6.660604e+05